In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

print(sys.path)


# from src.setup_problem_loss_custom_obj import Setup
from src.misc import visualize_result

import cvxpy as cp
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from src.misc import *

['/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1', '/home/onoue/ws/lukasiewicz_1']


In [ ]:
def construct_objective_function(obj, c1, c2_dict):
    """
    目的関数を構成する．
    c1 は logical constraints を，
    c2 は consistency constraints を
    満足する度合いを表す．
    """

    function = 0

    for j in range(obj.len_j):
        w = obj.w_j[j]
        function += 1/2 * (cp.norm2(w) ** 2)

    for j in range(obj.len_j):
        for l in range(obj.len_l):
            xi = obj.xi_jl[j, l]
            function += c1 * xi

    print(f'c1: {c1}')

    # for h in range(obj.len_h):
    #     xi = obj.xi_h[h, 0]
    #     function += c2 * xi

    df = obj.KB_info
    groups = set(df['group'].to_list())

    if groups != set(c2_dict.keys()):
        raise ValueError("The keys of c2_dict do not match the groups.")

    for group in groups:
        target_index = df[df['group'] == group].index.tolist()

        xi = [target_index]
        c2_weighted = c2_dict[group]
        function += c2_weighted * cp.norm(xi)

        print(f'c2: {c2_weighted}')

    obj.objective_function = cp.Minimize(function)

    return obj.objective_function

In [43]:
from src.setup_problem_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 10, 2: 2.5}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.19304633140563965 seconds! 

Loading rules ...
Done in 0.0019354820251464844 seconds! 

Identifying predicates ...
Done in 0.0017559528350830078 seconds! 

Constructing objective function ...
c1: 10
c2: 10
c2: 2.5
Done in 12.541025876998901 seconds! 

Constructing constraints ...
Done in 34.8726007938385 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 02:35:05 PM: Your problem has 12012 variables, 38016 constraints, and 0 parameters.
(CVXPY) Dec 03 02:35:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 02:35:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 02:35:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 03 02:35:25 PM: Compiling problem (target solver=ECOS).
(CVXPY) Dec 03 02:35:25 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[ 4.48954228e-09  6.96027694e-10 -1.24116352e-09  9.40354089e-11
  5.68636346e-10  1.38139392e-08  2.32346205e-10 -2.66627185e-10]

Accuracy: 
0.6716417910447762

Confusion matrix: 
[[90  0]
 [44  0]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.67      1.00      0.80        90
         1.0       0.00      0.00      0.00        44

    accuracy                           0.67       134
   macro avg       0.34      0.50      0.40       134
weighted avg       0.45      0.67      0.54       134


auc: 
0.7893939393939394


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [44]:
from src.setup_problem_loss_custom_obj import Setup

data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_4']
}

c2_dict = {1: 0.01, 2: 0.01}

obj_constructor = construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2_dict=c2_dict)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')


print()
print()
print()


test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

print("Outcome's coefficients: ")
print(problem_instance.w_j.value[3])
print()

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)
print()

roc_auc = roc_auc_score(y_test, y_pred)
print("auc: ")
print(roc_auc)

Loading data ...
Done in 0.13615202903747559 seconds! 

Loading rules ...
Done in 0.0030782222747802734 seconds! 

Identifying predicates ...
Done in 0.002865314483642578 seconds! 

Constructing objective function ...
c1: 10
c2: 0.01
c2: 0.01
Done in 6.4058144092559814 seconds! 

Constructing constraints ...
Done in 8.879814624786377 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 03 02:43:12 PM: Your problem has 12012 variables, 26224 constraints, and 0 parameters.
(CVXPY) Dec 03 02:43:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 03 02:43:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 03 02:43:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 03 02:43:22 PM: Compiling problem (target solver=ECOS).
(CVXPY) Dec 03 02:43:22 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Outcome's coefficients: 
[3.81647826e-07 6.98578875e-06 4.41734707e-06 1.41139079e-06
 1.98033245e-06 3.13172969e-07 1.98417213e-06 4.36615186e-07]

Accuracy: 
0.6716417910447762

Confusion matrix: 
[[90  0]
 [44  0]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.67      1.00      0.80        90
         1.0       0.00      0.00      0.00        44

    accuracy                           0.67       134
   macro avg       0.34      0.50      0.40       134
weighted avg       0.45      0.67      0.54       134


auc: 
0.8393939393939394


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [50]:
np.unique(problem_instance.xi_jl.value[3, :])

array([0.])

In [56]:
problem_instance.w_j[3].value

array([3.81647826e-07, 6.98578875e-06, 4.41734707e-06, 1.41139079e-06,
       1.98033245e-06, 3.13172969e-07, 1.98417213e-06, 4.36615186e-07])